In [10]:
import torch
import random
import os
import soundfile as sf
from tqdm import tqdm
import tarfile

# ===========================
# Устройство и параметры
# ===========================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используем устройство: {device}")

sample_rate = 48000
speaker_id = 'b_mhr'

# ===========================
# Загрузка модели Silero v4
# ===========================

print("Загружаем Silero v4...")
model, text = torch.hub.load(repo_or_dir='snakers4/silero-models', model= 'silero_tts',language='cyrillic', speaker='v4_cyrillic', trust_repo=True )

print("Модель Silero v4 успешно загружена!")
print(model)

# ===========================
# Шаблоны и слова для коротких предложений
# ===========================
templates = [
    "Мый {person} улат? {action}.",
    "{animal} кугу {location} шке.",
    "{food} {cook_method} паша тудо.",
    "{season} природа {state}, деревья {action}.",
    "{vehicle} по {road} влак.",
    "В {building} {meeting}, обсуждают {theme}.",
    "{month} {holiday}, дети играют в {game}.",
]

words = {
    'person': ['Иван', 'Мария', 'Алексей', 'Елена', 'Дмитрий'],
    'animal': ['Кот', 'Собака', 'Лиса', 'Заяц', 'Птица'],
    'location': ['лесу', 'полю', 'деревне', 'городу', 'реке'],
    'food': ['Суп', 'Пирог', 'Салат', 'Борщ', 'Плов'],
    'cook_method': ['плите', 'духовке', 'мангале', 'пароварке'],
    'season': ['весной', 'летом', 'осенью', 'зимой'],
    'state': ['прекрасна', 'оживает', 'меняет цвета', 'спит'],
    'action': ['цветут', 'шуршат', 'гнутся', 'покрыты снегом'],
    'vehicle': ['Автомобиль', 'Поезд', 'Самолет', 'Велосипед'],
    'road': ['трассе', 'шоссе', 'дороге', 'мосту'],
    'building': ['школе', 'офисе', 'театре', 'библиотеке'],
    'meeting': ['совещание', 'конференция', 'семинар'],
    'theme': ['проекте', 'планах', 'результатах'],
    'month': ['январе', 'марте', 'июне', 'октябре'],
    'holiday': ['праздник', 'выходные', 'отдых', 'поездка'],
    'game': ['футбол', 'шахматы', 'прятки', 'мячик'],
}

long_words = [
    "Мый", "тыге", "ош", "куте", "ула", "кодаш", "кугу", "влак", "поро", "шке",
    "Марий", "народ", "говорит", "язык", "сегодня", "погода", "день", "ночь",
    "солнце", "дождь", "ветер", "лес", "река", "озеро", "дом", "семья", "друзья"
]

# ===========================
# Генерация предложений
# ===========================
def generate_mari_sentences(total=9050):
    sentences = []
    for _ in tqdm(range(total), desc="Генерация предложений"):
        if random.random() < 0.6:
            template = random.choice(templates)
            sentence = template.format(**{k: random.choice(v) for k, v in words.items()})
        else:
            length = random.randint(20, 70)
            sentence = " ".join(random.choices(long_words, k=length)) + "."
        sentences.append(sentence)
    return sentences

print("\nГенерируем 9000+ предложений на марийском...")
sentences = generate_mari_sentences(9050)
print(f"Сгенерировано: {len(sentences)} предложений")

with open("texts.txt", "w", encoding="utf-8") as f:
    for s in sentences:
        f.write(s + "\n")
print("Файл texts.txt сохранён.")

# ===========================
# Синтез всех предложений
# ===========================
os.makedirs("audio", exist_ok=True)
success = 0
failed = []

print("\nНачинаем синтез полного датасета...")

for i, text in enumerate(tqdm(sentences, desc="Синтез речи")):
        audio = model.apply_tts(text=text, speaker=speaker_id, sample_rate=sample_rate)
        sf.write(f"audio/{i+1}.wav", audio.cpu().numpy(), sample_rate)
        success += 1


print(f"\nСинтез завершён. {success}/{len(sentences)} файлов успешно создано.")
if failed:
    print(f"Ошибки в строках: {failed[:10]} ... (первые 10)")

# ===========================
# Архивация датасета
# ===========================
archive_name = "bvt2202_gorshkova_7_silero_mhr_v4.tar.gz"

with tarfile.open(archive_name, "w:gz") as tar:
    tar.add("texts.txt", arcname="texts.txt")
    for file in tqdm(os.listdir("audio"), desc="Упаковка"):
        if file.endswith(".wav"):
            tar.add(f"audio/{file}", arcname=f"audio/{file}")

print(f"\nАрхив создан: {archive_name}")
print("Готово!\n")


Используем устройство: cpu
Загружаем Silero v4...


Using cache found in C:\Users\asus/.cache\torch\hub\snakers4_silero-models_master


Модель Silero v4 успешно загружена!
<<torch_package_7>.multi_acc_v3_package.TTSModelMultiAcc_v3 object at 0x0000016A95C5FF40>

Генерируем 9000+ предложений на марийском...


Генерация предложений: 100%|██████████| 9050/9050 [00:00<00:00, 135069.04it/s]


Сгенерировано: 9050 предложений
Файл texts.txt сохранён.

Начинаем синтез полного датасета...


Синтез речи: 100%|██████████| 9050/9050 [39:55<00:00,  3.78it/s]  



Синтез завершён. 9050/9050 файлов успешно создано.


Упаковка: 100%|██████████| 9050/9050 [08:03<00:00, 18.73it/s]


Архив создан: bvt2202_gorshkova_7_silero_mhr_v4.tar.gz
Готово!

